# Tutorial 1: using a Quantum Device to extract machine-learning features

(download this tutorial [here](https://github.com/pasqal-io/quantum-evolution-kernel/blob/main/examples/tutorial%201%20-%20Using%20a%20Quantum%20Device%20to%20Extract%20Machine-Learning%20Features.ipynb))

The companion library is a machine-learning tool that uses a quantum device to predict similarities and properties of graphs. In this tutorial, we will reproduce first part of the [QEK paper](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.107.042615) using the library's high-level API. The high-level goal of this tutorial is to predict toxicity properties of molecules, using quantum machine learning, but of course, the mechanisms involved are much more generic.

By the end of this notebook, you will know how to:

1. Setup import for a molecular dataset (the library supports other type of graphs, of course).
2. Setup compilation and execution of these graphs for execution on a Quantum Device (either an emulator or a physical QPU).
3. Launch the execution and extract the relevant machine-learning features.

A [companion notebook](./tutorial%202%20-%20Machine-Learning%20with%20the%20Quantum%20EvolutionKernel.ipynb) will guide you through machine-learning with QEK.

If, instead of using the library's high-level API, you prefer digging a bit closer to the qubits, you may prefer the companion [low-level notebook](./tutorial%201a%20-%20Using%20a%20Quantum%20Device%20to%20Extract%20Machine-Learning%20Features%20-%20low-level.ipynb) that mirrors this notebook, but using a lower-level API that will let you experiment with different quantum pulses.

## Dataset preparation

As in any machine learning task, we first need to load and prepare data. QEK can work with many types of graphs, including molecular graphs. For this tutorial, we will use the PTC-FM dataset, which contains such molecular graphs, labeled with their toxicity.

In [ ]:
# Ignore warnings for this tutorial.
import logging
import os
import sys

logger = logging.getLogger()
logger.disabled = True

sys.stderr = open(os.devnull, 'w')

In [ ]:
import torch_geometric.datasets as pyg_dataset

from qek.shared.retrier import PygRetrier

# Load the original PTC-FM dataset.
# We use PygRetrier to retry the download if it fails.
og_ptcfm = PygRetrier().insist(pyg_dataset.TUDataset, root="dataset", name="PTC_FM")

display("Loaded %s samples" % (len(og_ptcfm), ))

To extract machine-learning features from our dataset, we will need to configure a feature extractor. This library provides several feature extractors to either make use of a physical quantum device (QPU), or a variety of emulators.

To configure a feature extractor, we will need to provide a _compiler_, whose task is to take a list of graphs, extract embeddings and compile these embeddings to _sequences of pulses_, the format that can be executed  by either a QPU or an emulator. For this tutorial, our dataset is composed of molecule graphs following the PTCFM conventions, so we will use the `PTCFMCompiler`:

In [ ]:
import qek.data.graphs as qek_graphs

compiler = qek_graphs.PTCFMCompiler()

This library also provides other compilers from a variety of graph formats.

# Creating and executing a feature extractor from an emulator

The easiest way to process a graph is to compile and execute it for an emulator. QEK is built on top of Pulser, which provides several emulators. The simplest of these emulators is the `QutipEmulator`, which QEK uses for the `QutipExtractor`:

In [ ]:
from pathlib import Path
import qek.data.extractors as qek_extractors

# Use the Qutip Extractor.
extractor = qek_extractors.QutipExtractor(
    # Once computing is complete, data will be saved in this file.
    path=Path("saved_data.json"),
    compiler=compiler
)

# Add the graphs using the compiler we've picked previously.
extractor.add_graphs(graphs=og_ptcfm)

# We may now compile them.
compiled = extractor.compile()
display("Compiled %s sequences" % (len(compiled), ))

As you can see, the number of sequences compiled is lower than the number of samples loaded. Some of this is due to limitations within the algorithm (not all graphs can be efficiently laid out for execution on a Quantum Device), while others are due to the limitations of the emulator we target (which at the time of this writing is limited to ~50 qubits).

We may now run the extraction on the emulator:

In [ ]:
# Limit the number of qubits for this run, for performance reasons.
# You can increase this value to higher number of qubits, but this
# notebook will take longer to execute and may run out of memory.
#
# On our test computer, the practical limit is around 10 qubits.
max_qubits = 5
processed_dataset = extractor.run(max_qubits=max_qubits).processed_data
display("Extracted features from %s samples"% (len(processed_dataset), ))

If you wish to extract features from more samples, feel free to increase the value of `MAX_QUBITS` above. However, you will soon run into limitations of a quantum emulator, and possibly crash this notebook. At this point, you have other options, such as using `EmuMPSExtractor` instead of `QutipExtractor`, a more recent emulator that features much better performance in most cases, or you can run the extraction on a physical QPU.

# Creating and executing a feature extractor on a physical QPU

Once you have checked that low qubit sequences provide the results you expect on an emulator, you will generally want to move to a QPU.
For this, you will need either physical access to a QPU, or an account with [PASQAL Cloud](https://docs.pasqal.cloud), which provides
you remote access to QPUs built and hosted by Pasqal. In this section, we'll see how to use the latter.

If you don't have an account, just skip to the next section!

In [ ]:
from pathlib import Path


HAVE_PASQAL_ACCOUNT = False # If you have a PASQAL Cloud account, fill in the details and set this to `True`.

if HAVE_PASQAL_ACCOUNT:
    # Use the QPU Extractor.
    extractor = qek_extractors.RemoteQPUExtractor(
        # Once computing is complete, data will be saved in this file.
        path=Path("saved_data.json"),
        compiler = compiler,
        project_id = "XXXX", # Replace this with your project id on the PASQAL Cloud
        username = "XXX",    # Replace this with your username on PASQAL Cloud
        password = None,     # Replace this with your password on PASQAL Cloud or enter it on the command-line
    )

    # Add the graphs, exactly as above.
    extractor.add_graphs(graphs=og_ptcfm)

    # We may now compile, exactly as above.
    compiled = extractor.compile()
    display("Compiled %s sequences" % (len(compiled), ))

    # Launch the execution.
    extracted = extractor.run()
    display("Work enqueued with ids %s" % (extractor.batch_ids, ))

    # ...and wait for the results.
    await extracted
    processed_data = extracted.processed_data
    display("Extracted features from %s samples"% (len(processed_data), ))

As you can see, the process is essentially identical to executing with an emulator. Note that, as of this
writing, the waiting line to access a QPU can be very long (typically several hours).

There are two main ways to deal with this:

1. `RemoteQPUExtractor` can be attached to an ongoing job from batch ids, so that you can resume your work
    e.g. after turning off your computer.
2. Pasqal CLOUD offers access to high-performance hardware-based emulators, with dramatically
    shorter waiting lines. For instance, in the snippet above, you may replace `RemoteQPUExtractor`
    with `RemoteEmuMPSExtractor` to use the emu-mps emulator.

See [the documentation](https://pqs.pages.pasqal.com/quantum-evolution-kernel/) for more details.

## ...or using the provided dataset

For this notebook, instead of spending hours running the simulator on your computer, we're going to skip
this step and load on we're going to cheat and load the results, which are conveniently stored in `ptcfm_processed_dataset.json`.

In [ ]:
import qek.data.processed_data as qek_dataset
processed_dataset = qek_dataset.load_dataset(file_path="ptcfm_processed_dataset.json")
print(f"Size of the quantum compatible dataset = {len(processed_dataset)}")

## A look at the results

We can check the geometry for one of the samples:

In [ ]:
dataset_example = processed_dataset[64]
dataset_example.draw_register()

In [ ]:
dataset_example.draw_pulse()

The results of executing the embedding on the Quantum Device are in field `state_dict`:

In [ ]:
display(dataset_example.state_dict)
print(f"Total number of samples: {sum(dataset_example.state_dict.values())}")

This dictionary represents an approximation of the quantum state of the device for this graph after completion of the algorithm.

- each of the keys represents one possible state for the register (which represents the graph), with each qubit (which represents a single node) being in state `0` or `1`;
- the corresponding value is the number of samples observed with this specific state of the register.

In this example, for instance, we can see that the state observed most frequently is `10000001010`, with 43/1000 samples.

Note: Since Quantum Devices are inherently non-deterministic, you will probably obtained different samples if you run this on a Quantum Device instead of loading the dataset.


## Machine learning-features

From the state dictionary, we derive as machine-learning feature the _distribution of excitation_. We'll use this in a second to define our kernel.

In [ ]:
dataset_example.draw_excitation()

Is this distribution of excitation connected to the toxicity of the molecule? To check this out, we'll need to perform some machine-learning engineering.

**Note** Of course, you could derive features completely unrelated to distribution of excitation.

# What now?

What we have seen so far covers the use of a Quantum Device to extract machine-learning features.

For the next step, we'll see [how to use these features for actual machine learning](./tutorial%202%20-%20Machine-Learning%20with%20the%20Quantum%20EvolutionKernel.ipynb).